In [7]:
import tensorflow as tf
import numpy as np
import pickle
from torch.utils.data import IterableDataset, DataLoader, get_worker_info


In [20]:
>>> model = tf.keras.Sequential()
>>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=1))
>>> # The model will take as input an integer matrix of size (batch,
>>> # input_length), and the largest integer (i.e. word index) in the input
>>> # should be no larger than 999 (vocabulary size).
>>> # Now model.output_shape is (None, 10, 64), where `None` is the batch
>>> # dimension.
>>> input_array = np.random.randint(1000, size=(32, 1))
>>> model.compile('rmsprop', 'mse')
>>> output_array = model.predict(input_array)
>>> print(output_array.shape)


1/1 [==============================] - 0s 40ms/step
(32, 1, 64)


In [22]:
output_array[0]

array([[-7.62717798e-03,  2.62365825e-02,  4.83096726e-02,
         3.49922106e-03,  4.45561744e-02,  4.20769304e-03,
         3.59562524e-02,  3.16233672e-02,  1.31273158e-02,
        -1.43897645e-02,  2.66341679e-02, -8.28067213e-03,
         4.65450920e-02,  2.81646363e-02,  4.21465151e-02,
        -4.70392779e-03,  1.91234089e-02,  2.91728042e-02,
        -7.22243637e-03,  6.20406866e-03,  7.63901323e-03,
         1.62975900e-02, -4.66045737e-02, -3.06449085e-03,
         3.55370082e-02, -2.67876312e-03, -7.55916908e-03,
         4.82646115e-02,  3.78613546e-03, -7.57617876e-03,
        -4.12671193e-02, -5.58657572e-03,  2.47048773e-02,
        -1.12726204e-02, -2.64335424e-04, -9.28055122e-03,
         3.52434255e-02, -4.12894413e-03,  3.75962295e-02,
        -2.23878510e-02, -2.92806271e-02, -4.51295152e-02,
        -1.09318718e-02,  7.24419951e-05,  4.47081961e-02,
        -4.40355800e-02,  2.07252540e-02, -4.64570522e-03,
         2.15801261e-02,  1.97724439e-02,  4.44864146e-0

In [3]:
for i in range(0, 64, 512):
    print(i)

0


In [8]:
class GetTriplePair(IterableDataset):
    def __init__(self, item_size, user_list, pair, shuffle, num_epochs):
        self.item_size = item_size
        self.user_list = user_list
        self.pair = pair
        self.shuffle = shuffle
        self.num_epochs = num_epochs

    def __iter__(self):
        self.example_size = self.num_epochs * len(self.pair)
        self.example_index_queue = deque([])
        self.seed = 0
        self.start_list_index = None
        self.num_workers = 1
        self.index = 0
        return self

    def __next__(self):
        if self.index >= self.example_size:
            raise StopIteration
        # If `example_index_queue` is used up, replenish this list.
        while len(self.example_index_queue) == 0:
            index_list = list(range(len(self.pair)))
            if self.shuffle:
                random.Random(self.seed).shuffle(index_list)
                self.seed += 1
            if self.start_list_index is not None:
                index_list = index_list[self.start_list_index::self.num_workers]

                # Calculate next start index
                self.start_list_index = (self.start_list_index + (self.num_workers - (len(self.pair) % self.num_workers))) % self.num_workers
            self.example_index_queue.extend(index_list)
        result = self._example(self.example_index_queue.popleft())
        self.index += self.num_workers
        return result

    def _example(self, idx):
        u = self.pair[idx][0]
        i = self.pair[idx][1]
        j = np.random.randint(self.item_size)
        while j in self.user_list[u]:
            j = np.random.randint(self.item_size)
        return u, i, j

In [11]:
with open('/Users/vuhoang/Documents/code_base/APR-PyTorch/preprocessed/ml-1m.pickle', 'rb') as f:
        dataset = pickle.load(f)
        user_size, item_size = dataset['user_size'], dataset['item_size']
        train_user_list, test_user_list = dataset['train_user_list'], dataset['test_user_list']
        train_pair = dataset['train_pair']

In [12]:
dataset = GetTriplePair(item_size, train_user_list, train_pair, True, 1000)


In [13]:
dataset

In [14]:
loader = DataLoader(dataset, batch_size=2000)


In [15]:
loader

In [19]:
print(list(loader))

NameError: name 'deque' is not defined